# ‘Personal coffee-cafe launching location recommendation in Mega-city, SEOUL, Korea’

* Executor : Seungwon (Ronald) Seo

* Capstone Final Assignment - Foursquare API based data retrieving, data sanitization, grouping, clustering work

### Capturing table data from web page by using BeautifulSoup python library

##### Import required libraries from Python for this assignemnt & Captured table row/column data

In [20]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

##### Imported extra library for further going

In [21]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Extra Libraries imported.')

Extra Libraries imported.


### Exploring Seoul downtown(=Seoul city-hall) map with latitude & longitude values with markers

##### It is time to use client ID & client secret key for leveraging Foursquare ID

In [22]:
CLIENT_ID = 'JYEGBJGW25UASLAVZJ5Z1W1AIDDENGZQZMZ5LOB3DBGEQEGB' # your Foursquare ID
CLIENT_SECRET = 'QO0UHALGE3O525SMZOVPEKIWI2HYW4LFBOJSKCJA3TWHLGCK' # your Foursquare Secret
VERSION = '20180606'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JYEGBJGW25UASLAVZJ5Z1W1AIDDENGZQZMZ5LOB3DBGEQEGB
CLIENT_SECRET:QO0UHALGE3O525SMZOVPEKIWI2HYW4LFBOJSKCJA3TWHLGCK


##### Regarding Seoul city geograpical coordinate will be obtained

In [23]:
address = 'seoul'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.5666791 126.9782914


##### Let's set 1500 radius and search query with 'food'

In [24]:
search_query = 'food'
radius = 15000
print(search_query + ' .... OK!')

food .... OK!


##### Here is the Foursquare location API for this use case

In [25]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JYEGBJGW25UASLAVZJ5Z1W1AIDDENGZQZMZ5LOB3DBGEQEGB&client_secret=QO0UHALGE3O525SMZOVPEKIWI2HYW4LFBOJSKCJA3TWHLGCK&ll=37.5666791,126.9782914&v=20180606&query=food&radius=15000&limit=300'

In [26]:
results = requests.get(url).json()

##### Let's get initial draft version of data from Foursquar API, JSON

In [27]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4baf1b6ff964a52098eb3be3,중구 남대문로 81,KR,서울특별시,대한민국,롯데백화점 본점,335,"[중구 남대문로 81 (롯데백화점 본점), 소공동, 서울특별시, 서울특별시, 045...","[{'label': 'display', 'lat': 37.56469500955332...",37.564695,126.981160,NaN,04533,서울특별시,Food Court (푸드코트),v-1559982361,NaN
1,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,4e36b8112271b2cf06847869,서울특별시 중구 명동,KR,Jung-gu,대한민국,NaN,230,"[서울특별시 중구 명동, 소공동, Jung-gu, Seoul, Korea, 대한민국]","[{'label': 'display', 'lat': 37.5650383516418,...",37.565038,126.976703,NaN,NaN,"Seoul, Korea",Street Food 명동8길,v-1559982361,NaN
2,"[{'id': '4bf58dd8d48988d1f5941735', 'name': 'G...",False,53e35010498e03e4b6af811f,중구 소공로 63 신세계백화점 본점 B1,KR,서울특별시,대한민국,NaN,724,"[중구 소공로 63 신세계백화점 본점 B1, 회현동, 중구, 서울특별시, 대한민국]","[{'label': 'display', 'lat': 37.56051317737115...",37.560513,126.980928,NaN,NaN,서울특별시,SHINSEGAE FOOD MARKET,v-1559982361,NaN
3,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,5c96286d28374e002c985cfe,NaN,KR,서울특별시,대한민국,NaN,700,"[종로1.2.3.4가동, 서울특별시, 서울특별시, 110-160, 대한민국]","[{'label': 'display', 'lat': 37.571484, 'lng':...",37.571484,126.983420,인사동,110-160,서울특별시,Sky 31 Food Avenue,v-1559982361,NaN
4,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",False,4d520bed9b27721ecd3fbe46,종로구 신문로1가 163 광화문오피시아 지하1층,KR,서울특별시,대한민국,NaN,432,"[종로구 신문로1가 163 광화문오피시아 지하1층, 사직동, 서울특별시, 서울특별시...","[{'label': 'display', 'lat': 37.56954208476891...",37.569542,126.974977,NaN,110-999,서울특별시,오렌지 Food,v-1559982361,NaN


##### Further data praparation and sanitization...

In [28]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered=dataframe_filtered.drop(['distance','formattedAddress','cc','country','labeledLatLngs','state'],axis=1)

##### Getting of famous venue names around Seoul city downtown (Some Korean words are in here)

In [29]:
dataframe_filtered.name

0                                     Food Court (푸드코트)
1                                      Street Food 명동8길
2                                 SHINSEGAE FOOD MARKET
3                                    Sky 31 Food Avenue
4                                              오렌지 Food
5     Shinsegae Department Store Food Hall (신세계백화점 푸드홀)
6                      Jongno 3-ga Street Food (종로3가포차)
7                           THE 100 FOOD TRUCK (더백푸드트럭)
8                                        김밥천국 food cafe
9                                     서울식품 / Seoul Food
10                              flower food (황칠가 플라워푸드)
11                              CJ Food World (CJ 푸드월드)
12                                      SSG FOOD MARKET
13                                       Food Cafe 김밥천국
14                                    Jaws Food (죠스떡볶이)
15                  Doosan Tower Food Court (두산타워 푸드코트)
16                                 Food Empire (푸드엠파이어)
17                 Mr. Kebab Itaewon Halal Food 

##### Having image map regarding Seoul based on geograpical information (latitude and longitude)

In [30]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Seoul
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='seoul',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##### By using Foursquare API, Seoul nearby venues will be derived.

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['address', 
                  'address Latitude', 
                  'address Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
seoul_venues = getNearbyVenues(names=dataframe_filtered['address'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )



중구 남대문로 81
서울특별시 중구 명동
중구 소공로 63 신세계백화점 본점 B1
nan
종로구 신문로1가 163 광화문오피시아 지하1층
중구 충무로1가 52-5
nan
용산구 신흥로20길 45
nan
중구 남대문시장4길 12-2
자하문로 10
중구 동호로 330
언주로 30길 57
서대문구 신촌역로 19-2
종로구 안국동 21-1
중구 장충단로 275
서대문구 신촌동 연세세브란스 본관 3층
이태원로 192
강남구 압구정로 165
N서울타워 Plaza 1F
마포구 성산동 515 난지도길 66
강남구 도산대로 442
Cheonggyecheon 40
nan
nan
강남구 언주로30길 21
장충단로7길 9-1
중구 태평로
종로구 동숭동
nan
서대문구 신촌로 83
중구 을지로5길 26
nan
nan
영등포구 국제금융로 10
양천구 목동동로 257
강남구 테헤란로 517
중구 신당2동 종하빌딩 A B
Lotte Mart
강남구 도산대로89길 7
nan
종로구 관수동
nan
nan
nan
서대문구 이화여대길 33
nan
성동구 행당동 한양대병원 동관 1층
강남구 학동로101길 33
nan


##### Figuring out for Seoul venues Dataframe numbers

In [33]:
print(seoul_venues.shape)
seoul_venues.head()

(4988, 7)


,address,address Latitude,address Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,중구 남대문로 81,37.564695,126.98116,청담스포피아,37.565317,126.981374,Golf Course
1,중구 남대문로 81,37.564695,126.98116,The Westin Chosun Seoul (웨스틴조선호텔),37.564378,126.980058,Hotel
2,중구 남대문로 81,37.564695,126.98116,Nanta Theater (난타극장),37.563688,126.983687,Theater
3,중구 남대문로 81,37.564695,126.98116,Arc N Book (아크앤북),37.565930,126.981370,Bookstore
4,중구 남대문로 81,37.564695,126.98116,Sushi Cho (스시 조),37.564491,126.979743,Sushi Restaurant


### Analyzing of venue info. around Seoul downtown area

##### With address index detailed venue info will be obtained (* address is written in Korean language)

In [34]:
seoul_venues.groupby('address').count()

,address Latitude,address Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
address,,,,,,
Cheonggyecheon 40,100,100,100,100,100,100
Lotte Mart,100,100,100,100,100,100
N서울타워 Plaza 1F,100,100,100,100,100,100
강남구 도산대로 442,100,100,100,100,100,100
강남구 도산대로89길 7,100,100,100,100,100,100
강남구 압구정로 165,100,100,100,100,100,100
강남구 언주로30길 21,100,100,100,100,100,100
강남구 테헤란로 517,100,100,100,100,100,100
강남구 학동로101길 33,100,100,100,100,100,100


##### Need to know about numbers of unique categories in Seoul venues

In [35]:
print('There are {} uniques categories.'.format(len(seoul_venues['Venue Category'].unique())))

There are 189 uniques categories.


#### Grouping rows by addresses and by taking the mean of the frequency of occurrence of each category

In [36]:
# one hot encoding
seoul_onehot = pd.get_dummies(seoul_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seoul_onehot['address'] = seoul_venues['address'] 

# move neighborhood column to the first column
fixed_columns = [seoul_onehot.columns[-1]] + list(seoul_onehot.columns[:-1])
seoul_onehot = seoul_onehot[fixed_columns]

seoul_onehot.head()

,address,African Restaurant,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Bossam/Jokbal Restaurant,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Bunsik Restaurant,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Janguh Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pie Shop,Pier,Pizza Place,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Samgyetang Restaurant,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Village,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,중구 남대문로 81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,중구 남대문로 81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,중구 남대문로 81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,중구 남대문로 81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

##### Figuring out the shape of dataframe...

In [37]:
seoul_onehot.shape

(4988, 190)

##### Grouping by address column, mean number of occurence frequency of each category is obtained and further categories is shown

In [40]:
seoul_grouped = seoul_onehot.groupby('address').mean().reset_index()
seoul_grouped

,address,African Restaurant,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Bossam/Jokbal Restaurant,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Bunsik Restaurant,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Janguh Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pie Shop,Pier,Pizza Place,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Samgyetang Restaurant,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Village,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Cheonggyecheon 40,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.040000,0.01,0.020000,0.00,0.000000,0.00,0.00,0.00,0.02,0.000000,0.020000,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.02,0.000000,0.00,0.000000,0.070000,0.0,0.00,0.00,0.040000,0.01,0.010000,0.00,0.050000,0.00,0.000000,0.000000,0.000000,0.01,0.00,0.01,0.010000,0.00,0.030000,0.00,0.000000,0.000000,0.00,0.01,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.010000,0.00,0.000000,0.00,0.000000,0.03,0.00,0.00,0.00,0.11,0.02,0.000000,0.000000,0.01,0.010000,0.000000,0.00,0.020000,0.00,0.0,0.00,0.090000,0.01,0.00,0.010000,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.030000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.01,0.01,0.000000,0.00,0.01,0.00,0.00,0.000000,0.000000,0.03,0.000000,0.02,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.01,0.000000,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.01,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.00,0.010000,0.00,0.00,0.00,0.03,0.00,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.010000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
1,Lotte Mart,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.000000,0.030000,0.00,0

In [39]:
seoul_grouped.shape

(36, 190)

##### Writing a function to sort the venues in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Setting up the Dataframe with Top10 venues by Seoul downtown area

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['address']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['address'] = seoul_grouped['address']

for ind in np.arange(seoul_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seoul_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cheonggyecheon 40,Hotel,Korean Restaurant,Café,Coffee Shop,BBQ Joint,Chinese Restaurant,Historic Site,Plaza,Theater,Dessert Shop
1,Lotte Mart,Hotel,Korean Restaurant,Café,Bookstore,Chinese Restaurant,Coffee Shop,Historic Site,Cosmetics Shop,BBQ Joint,Bakery
2,N서울타워 Plaza 1F,Korean Restaurant,Café,Hotel,Theater,Gym / Fitness Center,Bakery,Park,Chinese Restaurant,Trail,Scenic Lookout
3,강남구 도산대로 442,Japanese Restaurant,Italian Restaurant,Korean Restaurant,Cocktail Bar,Coffee Shop,BBQ Joint,Sushi Restaurant,Café,Chinese Restaurant,Whisky Bar
4,강남구 도산대로89길 7,Coffee Shop,BBQ Joint,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Korean Restaurant,French Restaurant,Whisky Bar,Café,Salon / Barbershop


### Cluster address with k-means method

##### Setting up the number of clusters (here is no=5 )

In [43]:
kclusters = 5

seoul_grouped_clustering = seoul_grouped.drop('address', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seoul_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 0, 0, 4, 4, 0, 0], dtype=int32)

##### Also, adding clustering labels as 'Cluster Label'
##### Caution : When using Join() here, 'how' parameter is set to be 'right' in order to avoid float error

In [44]:
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

seoul_merged = dataframe_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
seoul_merged = seoul_merged.join(neighborhoods_venues_sorted.set_index('address'), on='address', how='right')
seoul_merged.head() # check the last columns!

,name,categories,address,city,crossStreet,lat,lng,neighborhood,postalCode,id,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Food Court (푸드코트),Food Court,중구 남대문로 81,서울특별시,롯데백화점 본점,37.564695,126.981160,NaN,04533,4baf1b6ff964a52098eb3be3,2,Hotel,Korean Restaurant,Café,Coffee Shop,Chinese Restaurant,Historic Site,Dessert Shop,Cosmetics Shop,Japanese Restaurant,Theater
1,Street Food 명동8길,Food Truck,서울특별시 중구 명동,Jung-gu,NaN,37.565038,126.976703,NaN,NaN,4e36b8112271b2cf06847869,2,Hotel,Korean Restaurant,Café,BBQ Joint,Coffee Shop,Chinese Restaurant,Theater,Dessert Shop,Historic Site,Plaza
2,SHINSEGAE FOOD MARKET,Gourmet Shop,중구 소공로 63 신세계백화점 본점 B1,서울특별시,NaN,37.560513,126.980928,NaN,NaN,53e35010498e03e4b6af811f,2,Hotel,Korean Restaurant,Café,Bakery,Historic Site,Noodle House,Chinese Restaurant,Dessert Shop,BBQ Joint,Coffee Shop
4,오렌지 Food,Korean Restaurant,종로구 신문로1가 163 광화문오피시아 지하1층,서울특별시,NaN,37.569542,126.974977,NaN,110-999,4d520bed9b27721ecd3fbe46,2,Hotel,Korean Restaurant,Coffee Shop,Café,Historic Site,Chinese Restaurant,Bookstore,Sushi Restaurant,History Museum,Japanese Restaurant
5,Shinsegae Department Store Food Hall (신세계백화점 푸드홀),Food Court,중구 충무로1가 52-5,서울특별시,본점 신관 11F,37.560250,126.981009,NaN,NaN,4baec72df964a5203cd73be3,2,Hotel,Korean Restaurant,Café,Bakery,BBQ Joint,Noodle House,Chinese Restaurant,Coffee Shop,Dessert Shop,Cosmetics Shop


##### Finally, creating map for clustering of Seoul downtown area

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seoul_merged['lat'], seoul_merged['lng'], seoul_merged['address'], seoul_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

##### Let's take a look at each different cluster and see what types of venues are already located. This is very important to make business deicitoin for setting up new coffee cafe hoping soft landing.

#### Cluster #1

In [46]:
seoul_merged.loc[seoul_merged['Cluster Label'] == 0, seoul_merged.columns[[1] + list(range(5, seoul_merged.shape[1]))]]

,categories,lat,lng,neighborhood,postalCode,id,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Burger Joint,37.547318,126.984102,후암동,04328,551bdd95498e5eba171ed8ef,0,Coffee Shop,Café,Korean Restaurant,Cocktail Bar,Park,Hotel,French Restaurant,BBQ Joint,Bistro,Scenic Lookout
17,Turkish Restaurant,37.534575,126.995407,NaN,04406,4b474179f964a520ae2d26e3,0,Coffee Shop,Korean Restaurant,Bakery,Lounge,Cocktail Bar,Bar,Dessert Shop,BBQ Joint,Vegetarian / Vegan Restaurant,Pizza Place
18,Grocery Store,37.527384,127.027303,NaN,135-724,4e893a797ee6eb4f4ee71d41,0,Korean Restaurant,Japanese Restaurant,Coffee Shop,BBQ Joint,Bakery,Sake Bar,Noodle House,Café,Chinese Restaurant,Sushi Restaurant
20,Food Court,37.566997,126.896041,NaN,121-250,4d8c71452da46ea8b432266d,0,Park,BBQ Joint,Korean Restaurant,Café,Coffee Shop,Italian Restaurant,Chinese Restaurant,Ice Cream Shop,Bakery,Harbor / Marina
21,Grocery Store,37.523410,127.044223,NaN,06062,4ff39df7e4b058e163995958,0,Japanese Restaurant,Italian Restaurant,Korean Restaurant,Cocktail Bar,Coffee Shop,BBQ Joint,Sushi Restaurant,Café,Chinese Restaurant,Whisky Bar
39,Bunsik Restaurant,37.525058,127.050541,NaN,06012,4cef89c362ef6dcb5ac0fe59,0,Coffee Shop,BBQ Joint,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Korean Restaurant,French Restaurant,Whisky Bar,Café,Salon / Barbershop
48,Bunsik Restaurant,37.522779,127.056755,NaN,135-957,52cbcd05498ee5349b284acc,0,Coffee Shop,Korean Restaurant,BBQ Joint,Japanese Restaurant,Chinese Restaurant,Italian Restaurant,Dessert Shop,Hotel,Park,Cantonese Restaurant


#### Cluster #2

In [47]:
seoul_merged.loc[seoul_merged['Cluster Label'] == 1, seoul_merged.columns[[1] + list(range(5, seoul_merged.shape[1]))]]

,categories,lat,lng,neighborhood,postalCode,id,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Food Court,37.563697,127.003272,광희동,100-400,4df182bfc65bf55ee52384c8,1,Korean Restaurant,Hotel,Noodle House,Coffee Shop,Market,Bakery,Chinese Restaurant,Dessert Shop,Park,Fried Chicken Joint
15,Food Court,37.568801,127.009280,NaN,100-851,4be3853d660ec9281977cc3b,1,Korean Restaurant,Coffee Shop,Hotel,Noodle House,Market,Bakery,Chinese Restaurant,BBQ Joint,Park,Indian Restaurant
26,Restaurant,37.562355,127.005806,장충동,NaN,5296c31e498e2a64e74a3471,1,Korean Restaurant,Hotel,Noodle House,Coffee Shop,Market,Bakery,Chinese Restaurant,Dessert Shop,Park,Indian Restaurant
28,Cosmetics Shop,37.563759,127.001591,NaN,NaN,4f2d0001e4b0620356176beb,1,Korean Restaurant,Hotel,Noodle House,Coffee Shop,Bakery,Park,Chinese Restaurant,Café,Market,Dessert Shop
37,Office,37.558269,127.007843,NaN,NaN,4bea4e7da9900f473b691640,1,Korean Restaurant,Hotel,Coffee Shop,Bakery,Noodle House,Theater,Dessert Shop,BBQ Joint,Market,Park
41,Food Truck,37.563845,127.010899,NaN,NaN,4e36b69c2271b2cf06846829,1,Korean Restaurant,Coffee Shop,Hotel,Noodle House,Chinese Restaurant,Market,Dessert Shop,Bakery,Park,Indian Restaurant


#### Cluster #3

In [48]:
seoul_merged.loc[seoul_merged['Cluster Label'] == 2, seoul_merged.columns[[1] + list(range(5, seoul_merged.shape[1]))]]

,categories,lat,lng,neighborhood,postalCode,id,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Food Court,37.564695,126.981160,NaN,04533,4baf1b6ff964a52098eb3be3,2,Hotel,Korean Restaurant,Café,Coffee Shop,Chinese Restaurant,Historic Site,Dessert Shop,Cosmetics Shop,Japanese Restaurant,Theater
1,Food Truck,37.565038,126.976703,NaN,NaN,4e36b8112271b2cf06847869,2,Hotel,Korean Restaurant,Café,BBQ Joint,Coffee Shop,Chinese Restaurant,Theater,Dessert Shop,Historic Site,Plaza
2,Gourmet Shop,37.560513,126.980928,NaN,NaN,53e35010498e03e4b6af811f,2,Hotel,Korean Restaurant,Café,Bakery,Historic Site,Noodle House,Chinese Restaurant,Dessert Shop,BBQ Joint,Coffee Shop
4,Korean Restaurant,37.569542,126.974977,NaN,110-999,4d520bed9b27721ecd3fbe46,2,Hotel,Korean Restaurant,Coffee Shop,Café,Historic Site,Chinese Restaurant,Bookstore,Sushi Restaurant,History Museum,Japanese Restaurant
5,Food Court,37.560250,126.981009,NaN,NaN,4baec72df964a5203cd73be3,2,Hotel,Korean Restaurant,Café,Bakery,BBQ Joint,Noodle House,Chinese Restaurant,Coffee Shop,Dessert Shop,Cosmetics Shop
9,Grocery Store,37.559365,126.977253,NaN,NaN,4ec7349d4690116a8bd27ee3,2,Korean Restaurant,Hotel,Coffee Shop,Historic Site,Café,BBQ Joint,Buffet,Park,Noodle House,Lounge
19,Korean Restaurant,37.551043,126.990892,NaN,NaN,51775e3ce4b077bc9874f670,2,Korean Restaurant,Café,Hotel,Theater,Gym / Fitness Center,Bakery,Park,Chinese Restaurant,Trail,Scenic Lookout
22,Korean Restaurant,37.565304,126.977883,NaN,100-180,4f79229de4b0e91fa8869e95,2,Hotel,Korean Restaurant,Café,Coffee Shop,BBQ Joint,Chinese Restaurant,Historic Site,Plaza,Theater,Dessert Shop
27,Korean Restaurant,37.563148,126.976466,NaN,NaN,4da023862305a35de60dc6af,2,Hotel,Korean Restaurant,Café,Chinese Restaurant,Bakery,Historic Site,Plaza,BBQ Joint,Dessert Shop,Noodle House
31,Greek Restaurant,37.567754,126.985180,NaN,100-210,51d8eac0498ef1b3a735c9a4,2,Korean Restaurant,Hotel,Café,Coffee Shop,Chinese Restaurant,Tea Room,Bookstore,Plaza,Pub,Dessert Shop


#### Cluster #4

In [49]:
seoul_merged.loc[seoul_merged['Cluster Label'] == 3, seoul_merged.columns[[1] + list(range(5, seoul_merged.shape[1]))]]

,categories,lat,lng,neighborhood,postalCode,id,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Japanese Curry Restaurant,37.576673,126.972650,NaN,03044,5ce285fb1f74400039cd48cb,3,Coffee Shop,Korean Restaurant,Café,Italian Restaurant,Art Gallery,History Museum,Historic Site,Japanese Restaurant,Sushi Restaurant,Bakery
13,Bunsik Restaurant,37.558530,126.942727,NaN,120-809,4b982966f964a5209a3035e3,3,Coffee Shop,Korean Restaurant,Dessert Shop,Bakery,Japanese Restaurant,BBQ Joint,Café,Dumpling Restaurant,Chinese Restaurant,Seafood Restaurant
14,Bunsik Restaurant,37.578772,126.982160,NaN,NaN,4ee589809adf39820089f5c1,3,Café,Coffee Shop,Tea Room,Italian Restaurant,Historic Site,Korean Restaurant,Art Gallery,Art Museum,Palace,Hotel
16,Food Court,37.562454,126.940807,NaN,120-749,4ef024ed9a52cecf42a04179,3,Coffee Shop,Café,Bakery,Chinese Restaurant,Dessert Shop,Dumpling Restaurant,Japanese Restaurant,BBQ Joint,Tea Room,Korean Restaurant
30,Grocery Store,37.555776,126.935697,NaN,NaN,4e812e73722e63cf9d3f1bb2,3,Coffee Shop,BBQ Joint,Korean Restaurant,Dessert Shop,Bakery,Café,Chinese Restaurant,Japanese Restaurant,Ramen Restaurant,Dumpling Restaurant
45,Snack Place,37.558292,126.945709,NaN,NaN,4eec8906b8f76a24a40cfe46,3,Coffee Shop,Bakery,Korean Restaurant,Café,BBQ Joint,Dumpling Restaurant,Dessert Shop,Seafood Restaurant,Japanese Restaurant,Beer Bar


#### Cluster #5

In [50]:
seoul_merged.loc[seoul_merged['Cluster Label'] == 4, seoul_merged.columns[[1] + list(range(5, seoul_merged.shape[1]))]]

,categories,lat,lng,neighborhood,postalCode,id,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Supermarket,37.490486,127.055323,NaN,NaN,5c0a6ff0dd8442002cdf2d09,4,Korean Restaurant,Coffee Shop,Bakery,BBQ Joint,Chinese Restaurant,Seafood Restaurant,Café,Noodle House,Italian Restaurant,Trail
25,Supermarket,37.488170,127.052297,NaN,06292,5789a787498edec74c507558,4,Korean Restaurant,Coffee Shop,Bakery,BBQ Joint,Café,Seafood Restaurant,Chinese Restaurant,Burger Joint,Bunsik Restaurant,Noodle House
34,Food Court,37.525420,126.925507,NaN,NaN,50402d89e4b09c6ce3d3c479,4,Korean Restaurant,Coffee Shop,Japanese Restaurant,Park,Noodle House,Hotel,BBQ Joint,Italian Restaurant,Ice Cream Shop,Bakery
35,Food Court,37.526636,126.875232,NaN,158-051,4ce4cb914303f04dee9595b8,4,Coffee Shop,Korean Restaurant,Bakery,Chinese Restaurant,Fast Food Restaurant,BBQ Joint,Sushi Restaurant,Dessert Shop,Ice Cream Shop,Donut Shop
36,Grocery Store,37.508483,127.060084,NaN,06164,4c037c78310fc9b69bfac561,4,Korean Restaurant,Coffee Shop,Hotel,Bakery,BBQ Joint,Gym / Fitness Center,Seafood Restaurant,Hotel Bar,Noodle House,Plaza
47,Food Court,37.559707,127.045178,NaN,NaN,4feace96e4b026bf4dcf6431,4,Coffee Shop,Korean Restaurant,BBQ Joint,Bakery,Café,Donut Shop,Ice Cream Shop,Seafood Restaurant,Vietnamese Restaurant,Shopping Mall


### end of code (pls back to report on github for further clarification and result discussion)